# ChatGPT chatlog:

Model: https://chatgpt.com/share/67fd0596-5db0-8011-990f-5b0f1d8d6f52 

User Interface: https://chatgpt.com/share/67fd0fd5-b7c4-8011-94e6-e02485efed51 

# Code:

For the best experience, run this script outside of Jupyter Notebook (e.g. PyCharm, Visual Studio Code). 
While the functionality remains the same, PyQt5 does not work well within Jupyter Notebook.

Issue: You may need to restart the kernel each time you run the code.

In [ ]:
# Install required packages (run in terminal if not already installed)

# pip install gurobipy pyqt5 pandas

In [ ]:
import sys
import pandas as pd
import math
from gurobipy import Model, GRB, quicksum
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QLineEdit, QCheckBox, QVBoxLayout, QGridLayout,
    QScrollArea, QSlider, QPushButton, QMessageBox, QGroupBox, QHBoxLayout,
    QTextEdit, QTableWidget, QTableWidgetItem, QTabWidget, QHeaderView, QLabel, QSpinBox
)
from PyQt5.QtCore import Qt

### LOP MODEL ###
def run_optimizer(selected_courses, max_consecutive_hours, blocked_times, alpha, beta, df, max_solutions = 3, filter_by_time=False):
    df_local = df[df["Course Code"].isin(selected_courses)].copy()
    df_local["Start"], df_local["End"] = zip(*df_local["TIME"].apply(lambda t: (int(t.split("-")[0]), int(t.split("-")[1]))))
    df_local["IndexID"] = df_local["Course Code"] + "_" + df_local["INDEX"].astype(str)

    courses = df_local["Course Code"].unique()
    index_ids = df_local["IndexID"].unique()
    days = df_local["DAY"].unique()

    index_course_map = df_local.drop_duplicates(subset=["IndexID"])[["IndexID", "Course Code"]].set_index("IndexID")["Course Code"].to_dict()
    index_intervals = {
        idx: list(zip(g["DAY"], g["Start"], g["End"]))
        for idx, g in df_local.groupby("IndexID")
    }

    model = Model("Timetable_Optimization")
    model.Params.OutputFlag = 0

    ### DV ###
    x = model.addVars(index_ids, vtype=GRB.BINARY, name="x")
    y = model.addVars(days, vtype=GRB.BINARY, name="y")
    z_start = model.addVars(days, vtype=GRB.INTEGER, lb=830, ub=2230)
    z_end = model.addVars(days, vtype=GRB.INTEGER, lb=830, ub=2230)
    z_total = model.addVars(days, vtype=GRB.INTEGER)

    # Constraints 1: Course Index Selection
    for course in courses:
        options = [idx for idx in index_ids if index_course_map[idx] == course]
        model.addConstr(quicksum(x[idx] for idx in options) == 1, name=f"one_index_for_{course}")

    # Constraints 3: Active School Day Tracking
    for idx in index_ids:
        for day, start, end in index_intervals[idx]:
            model.addConstr(x[idx] <= y[day], name=f"use_day_if_{idx}_{day}")

    # Constraints 2: No Overlapping Classes
    for i, idx1 in enumerate(index_ids):
        for j in range(i + 1, len(index_ids)):
            idx2 = index_ids[j]
            for d1, s1, e1 in index_intervals[idx1]:
                for d2, s2, e2 in index_intervals[idx2]:
                    if d1 == d2 and max(s1, s2) < min(e1, e2):
                        model.addConstr(x[idx1] + x[idx2] <= 1, name=f"overlap_{idx1}_{idx2}")
                        break

    # Constraints 4: Total Time in School per Day
    for d in days:
        for idx in index_ids:
            for day, start, end in index_intervals[idx]:
                if day == d:
                    model.addConstr(z_start[d] <= start + 2000 * (1 - x[idx]))
                    model.addConstr(z_end[d] >= end * x[idx])
        model.addConstr(z_total[d] == z_end[d] - z_start[d])

    # Constraints 5: Blocked Time Enforcement 
    for d in blocked_times:
        if not blocked_times[d]:
            model.addConstr(y[d] == 0, name=f"block_day_{d}")
        else:
            for t in blocked_times[d]:
                for idx in index_ids:
                    for day, start, end in index_intervals[idx]:
                        if day == d and start <= t < end:
                            model.addConstr(x[idx] == 0, name=f"block_time_{d}_{t}")

    # Constraints 6: Max Consecutive Hours
    time_slots = list(range(830, 2230, 100))
    class_at = model.addVars(days, time_slots, vtype=GRB.BINARY, name="class_at")

    for d in days:
        for t in time_slots:
            model.addConstr(
                class_at[d, t] == quicksum(
                    x[idx] for idx in index_ids
                    for day, start, end in index_intervals[idx]
                    if day == d and start <= t < end
                ) , name=f"class_at_{d}_{t}"
            )

    for d in days:
        for i in range(len(time_slots) - max_consecutive_hours):
            time_window = time_slots[i:i + max_consecutive_hours + 1]
            model.addConstr(quicksum(class_at[d, t] for t in time_window) <= max_consecutive_hours, name=f"max_consec_hours_{d}_{time_window[0]}")

    # use the of '700' stated in report section 3.3.1
    alpha_weight = 700 * alpha
    beta_weight = beta
    model.setObjective(alpha_weight * quicksum(y[d] for d in days) + beta_weight * quicksum(z_total[d] for d in days), GRB.MINIMIZE)

    # We want SUBOPTIMAL solution, we dont mind how bad the solution is. 
    # Pool Solution will be replace with a for loop to ensure the remove duplicate timeslot feature
    model.setParam(GRB.Param.PoolSearchMode, 2)
    model.setParam(GRB.Param.PoolGap, 1)
    # model.setParam(GRB.Param.PoolSolutions, 10)

    weekdays = ["MON", "TUE", "WED", "THU", "FRI"]
    unique_solutions = []
    seen_signatures = set()
    max_unique = max_solutions
    attempts = 0

    # use a large number, 50, to loop through and get max_unique amount of unique solutions
    while len(unique_solutions) < max_unique and attempts < 50:
        model.optimize()
        attempts += 1

        if model.Status not in [GRB.OPTIMAL, GRB.SUBOPTIMAL]:
            break

        solution_x = {idx: int(round(x[idx].X)) for idx in index_ids}

        if filter_by_time:
            occupied_slots = set()
            for idx, val in solution_x.items():
                if val == 1:
                    for day, start, end in index_intervals[idx]:
                        for t in range(start, end, 100):  # e.g., 0830, 0930
                            occupied_slots.add((day, t))
            signature = frozenset(sorted(occupied_slots))  # structural fingerprint
        else:
            signature = frozenset(idx for idx, val in solution_x.items() if val == 1)

        # Exclude current solution from future iterations
        model.addConstr(
            quicksum(
                (1 - x[idx]) if val == 1 else x[idx]
                for idx, val in solution_x.items()
            ) >= 1,
            name=f"exclude_solution_{len(unique_solutions)}"
        )
        if signature in seen_signatures:
            continue

        seen_signatures.add(signature)
        unique_solutions.append(solution_x)

    solutions = []
    stats_list = []

    for i, sol in enumerate(unique_solutions):
        result = []
        for idx, val in sol.items():
            if val == 1:
                course_code, index = idx.split("_", 1)
                rows = df_local[(df_local["Course Code"] == course_code) & (df_local["INDEX"].astype(str) == index)]
                for _, row in rows.iterrows():
                    result.append(row.to_dict())
        timetable_df = pd.DataFrame(result).sort_values(by=["DAY", "TIME"])
        solutions.append(timetable_df)

        # Course Summary Header
        course_summary = "📚 Selected Courses:\n"
        grouped = timetable_df.groupby(["Course Code", "INDEX", "GROUP"])

        for (code, index, group), rows in grouped:
            course_summary += f" - {code}, Index {index}, Group {group}\n"

        # start summary with course info first
        summary = course_summary + "\n"

        # Summary statistics recomputed per timetable
        active_days = sorted(timetable_df["DAY"].unique())
        summary += f"📅 Active School Days (Solution {i + 1}):\n"
        for d in weekdays:
            summary += f" - {d}: {1 if d in active_days else 0}\n"

        summary += "\n🕒 Time in school each Day:\n"
        total_time = 0
        for d in weekdays:
            daily_classes = timetable_df[timetable_df["DAY"] == d]
            if not daily_classes.empty:
                times = []
                for t in daily_classes["TIME"]:
                    try:
                        start, end = map(int, t.split("-"))
                        times.append((start, end))
                    except:
                        continue
                if times:
                    earliest = min(t[0] for t in times)
                    latest = max(t[1] for t in times)
                    timespent = (latest - earliest)
                    timespenthrs = max(0, int(math.ceil(timespent / 100)))
                    total_time += timespenthrs
                    summary += f" - {d}: {timespenthrs} hrs\n"
                else:
                    summary += f" - {d}: 0 hrs\n"
            else:
                summary += f" - {d}: 0 hrs\n"

        summary += f"\n⏳ Total Time in school: {total_time} hrs\n"
        summary += f"\n🎯 Solution {i + 1} of {len(unique_solutions)}\n"
        stats_list.append(summary)

    if model.status == GRB.INFEASIBLE:
        summary = "\n❌ Model is infeasible.\n"
        model.computeIIS()
        model.write("infeasible.ilp")  # Optional file for manual inspection
        model.Params.OutputFlag = 1

        infeasible_constraints = {c.constrName for c in model.getConstrs() if c.IISConstr}
        infeasible_bounds = {v.VarName for v in model.getVars() if v.IISLB or v.IISUB}

        if infeasible_constraints:
            summary += "\n🔒 Infeasible Constraints:\n"
            for name in sorted(infeasible_constraints):
                summary += f"  - {name}\n"

        if infeasible_bounds:
            summary += "\n📉 Infeasible Variable Bounds:\n"
            for name in sorted(infeasible_bounds):
                summary += f"  - {name}\n"

        stats_list.append(summary)

    return solutions, stats_list


# -----------------------------
# Load Data
# -----------------------------
file_path = "bus_clean.csv"
df = pd.read_csv(file_path)
df["Course Code"] = df["Course Code"].str.upper().str.strip()
df["DAY"] = df["DAY"].str.upper().str.strip()
df["TIME"] = df["TIME"].astype(str).str.strip()

all_courses = df["Course Code"].unique()
days_of_week = ["MON", "TUE", "WED", "THU", "FRI"]

class PreferenceSelector(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("\U0001F9E0 Best Timetable Preference Selector")
        self.setGeometry(100, 100, 1600, 900)

        self.setStyleSheet("""
            QWidget { font-size: 12pt; }
            QGroupBox {
                border: 1px solid #cccccc;
                border-radius: 10px;
                padding: 10px;
                margin-top: 5px;
            }
            QPushButton {
                background-color: #2ecc71;
                color: white;
                padding: 10px;
                border-radius: 10px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #27ae60;
            }
            QTableWidget {
                gridline-color: #555;
                font-size: 12pt;
            }
        """)

        main_layout = QHBoxLayout()
        left_layout = QVBoxLayout()
        right_layout = QVBoxLayout()
        main_layout.setSpacing(5)
        main_layout.setContentsMargins(10, 5, 10, 5)

        left_layout.setSpacing(5)
        left_layout.setContentsMargins(10, 5, 10, 5)

        right_layout.setSpacing(5)
        right_layout.setContentsMargins(10, 5, 10, 5)

        # Course Selection
        course_group = QGroupBox("\U0001F4DA Search & Select Your Courses")
        course_layout = QVBoxLayout()
        course_layout.setSpacing(5)
        course_layout.setContentsMargins(5, 5, 5, 5)
        self.search_bar = QLineEdit()
        self.search_bar.setPlaceholderText("Search courses...")
        self.search_bar.textChanged.connect(self.filter_courses)
        course_layout.addWidget(self.search_bar)

        self.course_area = QScrollArea()
        self.course_area.setWidgetResizable(True)
        self.course_container = QWidget()
        self.course_grid = QGridLayout(self.course_container)
        self.course_area.setFixedHeight(250)
        self.course_grid.setAlignment(Qt.AlignTop)
        self.course_checkboxes = {}

        for idx, course in enumerate(all_courses):
            checkbox = QCheckBox(course)
            self.course_grid.addWidget(checkbox, idx, 0)
            self.course_checkboxes[course] = checkbox

        self.course_area.setWidget(self.course_container)
        course_layout.addWidget(self.course_area)
        course_group.setLayout(course_layout)
        left_layout.addWidget(course_group)

        # Max Consecutive Hours
        left_layout.addWidget(self.build_slider_group("\u23F1 Max Consecutive Hours", 4, 13, 6, "max_hours_label", "max_hours_slider", self.update_max_hours))

        # Blocked Times
        blocked_group = QGroupBox("\u274C Block Out Specific Days & Time Slots")
        grid_layout = QGridLayout()
        self.blocked_times = {}
        for i, day in enumerate(days_of_week):
            grid_layout.addWidget(QLabel(day), i, 0)
            self.blocked_times[day] = []
            for t in [830, 930, 1030, 1130, 1230, 1330, 1430, 1530, 1630, 1730, 1830, 1930, 2030, 2130, 2230]:
                checkbox = QCheckBox(str(t))
                grid_layout.addWidget(checkbox, i, (t // 100 - 8) + 1)
                self.blocked_times[day].append(checkbox)
        blocked_group.setLayout(grid_layout)
        left_layout.addWidget(blocked_group)

        # Alpha and Beta Sliders
        left_layout.addWidget(self.build_slider_group("\U0001F4C5 Prioritize Days in School (Alpha)", 1, 10, 5, "alpha_label", "alpha_slider", self.update_alpha))
        left_layout.addWidget(self.build_slider_group("\u23F1 Prioritize Total Time Spent (Beta)", 1, 10, 5, "beta_label", "beta_slider", self.update_beta))


        solution_group = QGroupBox("🔢 Number of Timetables to Generate")
        solution_layout = QVBoxLayout()
        self.solution_spin = QSpinBox()
        self.solution_spin.setMinimum(1)
        self.solution_spin.setMaximum(10)
        self.solution_spin.setValue(3)
        self.solution_spin.setSuffix(" solutions")
        solution_layout.addWidget(self.solution_spin)
        solution_group.setLayout(solution_layout)
        left_layout.addWidget(solution_group)

        self.same_slot_filter = QCheckBox("Treat Same Timetable (Different Index) as Duplicate")
        self.same_slot_filter.setChecked(False)
        left_layout.addWidget(self.same_slot_filter)

        # Submit Button
        submit_btn = QPushButton("\U0001F680 Submit & Optimize")
        submit_btn.clicked.connect(self.submit_preferences)
        left_layout.addWidget(submit_btn)

        # Output Tabs
        self.tab_widget = QTabWidget()
        self.stats_box = QTextEdit()
        self.stats_box.setReadOnly(True)
        right_layout.addWidget(QLabel("\U0001F4CB Timetable Solutions"))
        right_layout.addWidget(self.tab_widget)
        right_layout.addWidget(QLabel("\U0001F4CA Summary Stats"))
        right_layout.addWidget(self.stats_box)

        main_layout.addLayout(left_layout, stretch=3)
        main_layout.addLayout(right_layout, stretch=5)
        self.setLayout(main_layout)

    def build_slider_group(self, label_text, min_val, max_val, default, label_attr, slider_attr, callback):
        group = QGroupBox()
        vbox = QVBoxLayout()
        vbox.setSpacing(2)
        vbox.setContentsMargins(2, 2, 2, 2)
        label = QLabel(f"{label_text}: {default}")
        slider = QSlider(Qt.Horizontal)
        slider.setMinimum(min_val)
        slider.setMaximum(max_val)
        slider.setValue(default)
        slider.valueChanged.connect(callback)

        setattr(self, label_attr, label)
        setattr(self, slider_attr, slider)

        vbox.addWidget(label)
        vbox.addWidget(slider)
        group.setLayout(vbox)
        return group

    def filter_courses(self, text):
        text = text.lower()
        for course, checkbox in self.course_checkboxes.items():
            checkbox.setVisible(text in course.lower())

    def update_max_hours(self):
        self.max_hours_label.setText(f"\u23F1 Max Consecutive Hours: {self.max_hours_slider.value()}")

    def update_alpha(self):
        self.alpha_label.setText(f"\U0001F4C5 Prioritize Days in School (Alpha): {self.alpha_slider.value()}")

    def update_beta(self):
        self.beta_label.setText(f"\u23F1 Prioritize Total Time Spent (Beta): {self.beta_slider.value()}")

    def submit_preferences(self):
        self.selected_courses = [c for c, cb in self.course_checkboxes.items() if cb.isChecked()]
        self.max_consecutive_hours = self.max_hours_slider.value()
        self.max_solutions = self.solution_spin.value()
        self.filter_by_time = self.same_slot_filter.isChecked()

        self.final_blocked_times = {}
        for day, checkboxes in self.blocked_times.items():
            blocked_times = [int(cb.text()) for cb in checkboxes if cb.isChecked()]
            if blocked_times:
                self.final_blocked_times[day] = blocked_times

        self.alpha = self.alpha_slider.value()
        self.beta = self.beta_slider.value()

        if not self.selected_courses:
            QMessageBox.warning(self, "Selection Error", "Please select at least one course.")
            return

        results, stats_list = run_optimizer(
            selected_courses=self.selected_courses,
            max_consecutive_hours=self.max_consecutive_hours,
            blocked_times=self.final_blocked_times,
            alpha=self.alpha,
            beta=self.beta,
            df=df,
            max_solutions=self.max_solutions,
            filter_by_time=self.filter_by_time
        )

        self.tab_widget.clear()
        # if not results:
        #     self.stats_box.setPlainText("❌ No feasible timetable could be found. Try adjusting your preferences.")
        #     return

        for i, timetable_df in enumerate(results):
            table = QTableWidget()
            table.setRowCount(14)
            table.setColumnCount(5)
            table.setHorizontalHeaderLabels(["MON", "TUE", "WED", "THU", "FRI"])
            table.setVerticalHeaderLabels([f"{h:02}:30" for h in range(8, 22)])
            table.setEditTriggers(QTableWidget.NoEditTriggers)
            table.setShowGrid(True)
            table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)

            time_row_map = {f"{h:02}30": i for i, h in enumerate(range(8, 22))}
            day_col_map = {"MON": 0, "TUE": 1, "WED": 2, "THU": 3, "FRI": 4}

            for _, row in timetable_df.iterrows():
                course = row["Course Code"]
                index = row["INDEX"]
                time_range = row["TIME"]
                day = row["DAY"]
                try:
                    start, end = map(int, time_range.split("-"))
                except:
                    continue
                for h in range(8, 22):
                    t = h * 100 + 30
                    if start <= t < end:
                        time_str = f"{h:02}30"
                        r = time_row_map[time_str]
                        c = day_col_map[day]
                        label = f"{course}\n({index})"
                        existing = table.item(r, c)
                        new_label = label if not existing else existing.text() + "\n" + label
                        item = QTableWidgetItem(new_label)
                        item.setTextAlignment(Qt.AlignCenter)
                        item.setToolTip(new_label)  # shows full text on hover

                        # Allow multi-line display
                        item.setData(Qt.DisplayRole, new_label)
                        table.setItem(r, c, item)

            self.tab_widget.addTab(table, f"Solution {i+1}")

        self.solution_stats = stats_list  # store all summaries
        self.stats_box.setPlainText(stats_list[0])  # default to first
        self.tab_widget.currentChanged.connect(self.update_summary_stats)

    def update_summary_stats(self, index):
        if hasattr(self, "solution_stats") and 0 <= index < len(self.solution_stats):
            self.stats_box.setPlainText(self.solution_stats[index])


app = QApplication(sys.argv)
window = PreferenceSelector()
window.show()
sys.exit(app.exec_())
